In [4]:
import matplotlib.pyplot as plt
import numpy as np

%matpotlib inline

In [55]:
# simulate data

In [50]:
PRESIDENT_PROBABILITY = .01
ACCIDENT_PROBABILITY = .1
TRIALS = 1000000

In [51]:
president = np.random.binomial(n=1, p=PRESIDENT_PROBABILITY, size=TRIALS)
accident = np.random.binomial(n=1, p=ACCIDENT_PROBABILITY, size=TRIALS)

In [52]:
print( 'President Mean: {}'.format(president.mean()) )
print( 'Accident Mean: {}'.format(accident.mean()) )

President Mean: 0.009838
Accident Mean: 0.100273


In [53]:
traffic_probabilities = []
for president_observed, accident_observed in zip(president, accident):
    if not president_observed:
        probability = .5 if accident_observed else .1
    else:
        probability = .9 if accident_observed else .6
    traffic_probabilities.append( probability )
    
traffic = np.random.binomial(n=1, p=traffic_probabilities)

In [54]:
print( 'Traffic Mean: {}'.format(traffic.mean()) )

Traffic Mean: 0.14487


In [ ]:
# probabilities are no longer fixed values, but distributions

In [66]:
times_president_observed = sum(president)
times_president_not_observed = len(president) - times_president_observed

president_probability_samples = np.random.beta(
    a=1 + times_president_observed, 
    b=1 + times_president_not_observed, 
    size=4567
)

In [70]:
times_accident_observed = sum(accident)
times_accident_not_observed = len(accident) - times_accident_observed

accident_probability_samples = np.random.beta(
    a=1 + times_accident_observed, 
    b=1 + times_accident_not_observed, 
    size=4567
)

In [71]:
# to compute things like P(y=1|a=1, p=0), well, it's logistic regression! 
# then make counterfactual plots to examine how probability varies with inputs